The sentiment of reviews is binary, meaning the review rating <=2 results in a sentiment score of 0, and rating >=4 have a sentiment score of 1. 

In [4]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('reviews.csv')
dataset.drop(columns="Id", inplace=True)
dataset

FileNotFoundError: ignored

In [5]:
dataset = dataset.drop_duplicates(ignore_index=True)
dataset

NameError: ignored

In [ ]:
dataset['Label'].hist()

In [ ]:
dataset['Label'].value_counts()

In [ ]:
# 어중간한 3점 삭제해 버리기
rating_3 = dataset[dataset['Label']==3].index
del_3 = dataset.drop(rating_3)
dataset=del_3

In [ ]:
dataset['Label'].value_counts()

In [ ]:
# 현재 4~5점(긍정)에 데이터가 너무 많음
#1~2점(부정)과 4~5점(긍정) 데이터 비율을 맞춰주기
def reduce_size(x):
    if len(x.index) > 70000:
        return x.sample(n=int(len(x.index)/35)) #70000/2200
    elif len(x.index) > 17000:
        return x.sample(n=int(len(x.index)/7)) #17000/2200
    else:
        return x

dataset = dataset.groupby('Label').apply(reduce_size).reset_index(drop=True)
dataset['Label'].value_counts()

# 4~5점 = 4601 / 1~2점 = 4622

In [ ]:
dataset.shape

In [ ]:
dataset

In [ ]:
# Ready to split 'train set' and  'test set'
X = dataset['Review']
y = dataset['Label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,  y,  test_size=0.5,shuffle=True,  random_state=1004)
             
print('X_train shape:', X_train.shape)

print('X_test shape:', X_test.shape)

print('y_train shape:', y_train.shape)

print('y_test shape:', y_test.shape)

In [ ]:
print(type(y_train))
print(type(X_test))

In [ ]:
# train과 test의 데이터에 각 점수들이 반반씩 잘 분포 되었음.
print('train set:', y_train.value_counts())
print('----------------------------')
print('test set:', y_test.value_counts())

In [ ]:
y

In [ ]:
dataset.dtypes

In [ ]:
#replace has a default arg inplace=False. 
#Thus, your results will not affect each other and you will have to combine them into dataset['label']=dataset['label'].replace([1,2,4,5],[0,0,1,1]) or 
#dataset['label'].replace([1,2,4,5],[0,0,1,1],inplace=True)

dataset['Label']=dataset['Label'].replace([1,2,4,5],[0,0,1,1])

In [ ]:
dataset['Label'].value_counts()

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
# concat()dms df와 series에 다 적용가능
train_set = pd.concat([X_train, y_train], axis = 1)
print(train_set)

In [ ]:
train_set.head(8)

In [ ]:
train_set['Review'][7]

In [ ]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train_set['Review'][7], "html5lib")
print(train_set['Review'][:20])


example1.get_text()[:10]

In [ ]:
import re
letters_only= re.sub('[^a-zA-Z]', ' ', example1.get_text())
letters_only[:10]

In [ ]:
# making lower case
lower_case = letters_only.lower()
# lower_case = letters_only['Review'].str.lower()      # Q str?

In [ ]:
# Tokenizing
words = lower_case.split()
print(len(words))
words[:10]

In [ ]:
# DOwnload for preprocessing the text
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')[:20]

Stopwords

In [ ]:
# Tokens exceot for stopwords 
words = [w for w in words if not w in stopwords.words('english')]
print(len(words))
words[:10]

## SnowballStemmer

print("The stemmed form of studying is: {}".format(stemmer.stem("studying")))  
print("The stemmed form of studies is: {}".format(stemmer.stem("studies")))  
print("The stemmed form of study is: {}".format(stemmer.stem("study")))  

>The stemmed form of studying is: studi  
>The stemmed form of studies is: studi  
>The stemmed form of study is: studi  

In [ ]:
# Before stemmer
words[:30]

In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

wordss = [stemmer.stem(w) for w in words]
# After stemmer
words[:10]

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

words = [wordnet_lemmatizer.lemmatize(w) for w in words]
# 처리 후 단어
words[:10]

In [ ]:
def review_to_words(raw_review ):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. 파이썬에서는 리스트보다 세트로 찾는게 훨씬 빠르다.
    # stopwords 를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    # 5. Stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words) )

In [ ]:
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라메터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [ ]:
%time clean_train_reviews = apply_by_multiprocessing(\
    train_set['Review'], review_to_words, workers=4)  

In [ ]:
clean_train_reviews

In [ ]:
%time clean_test_reviews = apply_by_multiprocessing(\
   X_test, review_to_words, workers=4)  

## Bag-of-words model - Wikipedia  
다음의 두 문장이 있다고 하자,  

(1) John likes to watch movies. Mary likes movies too.  
(2) John also likes to watch football games.  
위 두 문장을 토큰화 하여 가방에 담아주면 다음과 같다.  

[
    "John",  
    "likes",  
    "to",  
    "watch",  
    "movies",  
    "Mary",    
    "too",  
    "also",  
    "football",  
    "games"  
]  
그리고 배열의 순서대로 가방에서 각 토큰이 몇 번 등장하는지 횟수를 세어준다.  
  
(1) [1, 2, 1, 1, 2, 1, 1, 0, 0, 0]    
(2) [1, 1, 1, 1, 0, 0, 0, 1, 1, 1]    
=> 머신러닝 알고리즘이 이해할 수 있는 형태로 바꿔주는 작업이다.  
  
단어 가방을 n-gram을 사용해 bigram 으로 담아주면 다음과 같다.  

[  
    "John likes",  
    "likes to",  
    "to watch",  
    "watch movies",  
    "Mary likes",  
    "likes movies",  
    "movies too",  
]  
=> 여기에서는 CountVectorizer를 통해 위 작업을 한다.  

### <CountVectorizer 파라미터>
출처: https://chan-lab.tistory.com/27 [은공지능 공작소]
1.  analyzer 파라미터는 학습단위를 결정하는 파라미터입니다.
- word, char 2가지 옵션 정도를 고려해볼 수 있습니다.
- analyzer = 'word'라고 설정시, 학습의 단위를 단어로 설정합니다. (ex - home, go, my ...)
- analyzer = 'char'라고 설정시, 학습의 단위를 글자로 설정합니다.(ex - a, b, c, d ...)
- char 기반의 analyzer을 하면, ('a', 1), ('b', 2), ('c', 3), ('e', 4), ('g', 5), ('h', 6), ('i', 7)... 형태로 출력되기 때문에 실제 모델에서
좋은 성능을 보일지 의문이 남을 수도 있을 것 같습니다.  
결론부터 말씀드리면, 상당히 좋은 결과를 내놓을 수 있습니다.  
제가 CNN 모델에서 char analyzer로 학습을 시킨 적이 있었는데,
생각보다 괜찮은 결과가 나왔던 것으로 기억합니다.  
그러니 모델의 성능을 올리기 위해서, word 기반 방법만 고집할 것이 아니라 char 기반의 방법도 시도해 볼 만하다고 생각합니다.    

2.min_df 파라미터 
- 최소 빈도값을 설정해주는 파라미터
최소 DF가 2로 설정되었으니, 1인 것들은 모두 탈락하게 됩니다.

3. ngram_range 파라미터   
- 만약 ngram_range = (1, 3)라고 한다면, 단어의 묶음을 1개부터 3개까지 설정하라는 뜻입니다.
- 단어사전에는 1개 단어묶음도 있고, 2개 단어묶음도, 3개 단어묶음도 존재하게 되겠죠.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2, # 토큰이 나타날 최소 문서 개수
                             ngram_range=(1, 3),
                             max_features = 20000
                            )
vectorizer

## pipeline

https://deepinsight.tistory.com/173


In [ ]:
# 여기에서는 하나의 과정만 묶어주어 pipeline이 불필요 할 수도 있습니다.
# pipeline은 feature engineering의 여러 과정을 묶어 줄 때 사용합니다.
pipeline = Pipeline([
    ('vect', vectorizer),
])  

In [ ]:
'''
CountVectorizer는 여러 메소드 목록을 제공하고 있습니다.
메소드 중에서 fit_transform 을 하면  term-document matrix 를 반환하는 것을 볼 수 있습니다.
파이프라인으로 fit_transform을 해주게 되는데 해당 과정의 결과로 피처의 목록과 등장여부를 알 수 있습니다.
'''
%time train_data_features = pipeline.fit_transform(clean_train_reviews)
train_data_features

In [ ]:
# vectorize를 한 결과 
train_data_features.shape

In [ ]:
'''
get_feature_names() 라는 것도 CountVectorizer 에서 사용할 수 있는 메소드 중 하나입니다.
위에서 정의한 CountVectorizer 에 데이터를 넣어주고 fit 혹은 fit_transform 을 하게 되면 벡터화와 함께 사전(get_feature_names)을 생성하게 됩니다.
'''
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

In [ ]:
# 벡터화 된 피처를 확인해 봄
import numpy as np
dist = np.sum(train_data_features, axis=0)
    
for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

In [ ]:
pd.DataFrame(train_data_features[:10].toarray(), columns=vocab).head()

## 예측

In [ ]:
# 체크해 보고 싶은 Review(string)를 Series에 담아준다.
# reveiw_sample1 = "I don't like to spend money to bad lecture, but Cousera MOOC is really helpful to upgrade my understanding of U.S history."

reveiw_sample1 = "I I don't understand "
sample1 = pd.Series(reveiw_sample1)

In [ ]:
# 테스트 데이터를 벡터화 함
%time test_data_feature = pipeline.transform(sample1)

test_data_feature = test_data_feature.toarray()

In [ ]:
test_data_feature

In [ ]:
# clean_train_reviews에 대한 vocab 
# 벡터화 하며 만든 사전에서 해당 단어가 무엇인지 찾아볼 수 있다.
# vocab = vectorizer.get_feature_names()
# vocab[8], vocab[2558], vocab[2559], vocab[2560]

In [ ]:
# 테스트 데이터를 넣고 예측한다.
result = model.predict(test_data_feature)
# result[:100] # 100개의 리뷰에 대해서 뽑을 때 
# 여기서는 test_data_feature 하나니까 결과값이 1개 나오는 것임 
print("결과:",result)  #긍정1 부정0

## 모델의 정확도를 높이기 위한 개선 방법
1. Stopwords 라이브러리 추가 
2. Convector 파라미터 바꿔가면서 해보기 